In [1]:
"""
Created on Thurs Jan 9 09:39:35 2020
Group 1
@authors: Aicha DIALLO, Fatima SECK
"""

'\nCreated on Thurs Jan 9 09:39:35 2020\nGroup 1\n@authors: Aicha DIALLO, Fatima SECK\n'

# Rome2rio

In [2]:
import ast
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import re

In [3]:
# Activation of Chrome Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
webdriver = "/home/sid2018-5/Téléchargements/chromedriver"

driver = Chrome(webdriver, chrome_options=chrome_options)

/home/sid2018-5/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


def split_it(days):
    i="days"
    if i in days : 
        c = int(((re.findall("[0-9]+"+" days", days))[0]).replace(jour,""))
    else: 
        c = 0
    return c

In [4]:
def getdata(departure_city, arrival_city):
    """Documentation

    Parameters:

        departure_city: The departure city
        arrival_city: The arrival city

    Return:

        datafinal: The final dataframe with the variables imposed in the data dictionary for rom2rio

    """

    # The link of the website we want to scrap
    url = "https://www.rome2rio.com/map/"+departure_city+"/"+arrival_city
    driver.get(url)

    # The number of items we have by class
    items = len(driver.find_elements_by_class_name("route__title"))
    items2 = len(driver.find_elements_by_class_name("route__details"))

    total = []
    data = pd.DataFrame()

    # driver.find_elements_by_class_name : to locate an element by class attribute name
    for i in range(items):
        a=driver.find_elements_by_class_name("route__title")[i].text
        b=driver.find_elements_by_class_name("route__details")[i].text

        ####nb days
        jour  = "days"
        if jour in a : 
            nb_days = int(((re.findall("[0-9]+"+" days", a))[0]).replace(jour,""))
        else: 
            nb_days = 0
        ####nb hours
            nb_hours  = (re.search("[0-9]+"+"h",a ))
            if nb_hours:
                nb_hours = int(((re.search("[0-9]+"+"h",a )).group(0)).replace("h",""))
            else : 
                nb_hours = 0

        ####nb minute
        nb_minute  = re.search("[0-9]+"+"m",a )
        if nb_minute :
            nb_minute = int((nb_minute.group(0)).replace("m",""))
        else : 
            nb_minute = 0
            
        new = ((a,b,nb_days,nb_hours,nb_minute))
        total.append(new)
    
    data = pd.DataFrame(total,columns=['itineraire','prix','liste_jour','liste_hours','liste_minute'])

    data['Data_Source'] = 'Rome2rio'
    data['Departure_city'] = departure_city
    data['Arrival_city'] = arrival_city
    data['Nb_bus_taken'] = data.itineraire.str.count("Bus|bus")
    data['Nb_train_taken'] = data.itineraire.str.count("Train|train")
    data['Nb_car_taken'] = data.itineraire.str.count("Drive|drive|Rideshare|rideshare|car|Car")
    data['Nb_plane_taken'] = data.itineraire.str.count("Fly|fly")

    data['Price_max'] = data['prix'].apply(lambda st: st[st.find("-")+1:])
    data['Price_min'] = data['prix'].apply(lambda st: st[:st.find("-")])
    data['Duration1'] = data['itineraire'].apply(lambda st: st[st.find("•")+1:])
        
    data['days']= data['liste_jour']
    data['hours']= data['liste_hours']
    data['minute']= data['liste_minute']
    
    data['days'] = data['days'].replace('','0')
    data['minute'] = data['minute'].replace('','0')
    data['hours'] = data['hours'].replace('','0')
    
    data['Duration'] = data['days']*24*60+data['hours']*60+data['minute']
    
    datafinal = data[['Data_Source', 'Departure_city', 'Arrival_city', 'Nb_bus_taken',
                      'Nb_train_taken', 'Nb_car_taken', 'Nb_plane_taken','Duration', 'Price_min', 'Price_max']]

    return datafinal

In [5]:
getdata("Berlin","London")

,Data_Source,Departure_city,Arrival_city,Nb_bus_taken,Nb_train_taken,Nb_car_taken,Nb_plane_taken,Duration,Price_min,Price_max
0,Rome2rio,Berlin,London,0,0,0,1,277,£103,£198
1,Rome2rio,Berlin,London,0,0,0,1,281,£82,£178
2,Rome2rio,Berlin,London,0,1,0,1,304,£41,£206
3,Rome2rio,Berlin,London,0,1,0,1,0,£51,£198
4,Rome2rio,Berlin,London,0,0,0,1,311,£47,£142
5,Rome2rio,Berlin,London,0,1,0,1,317,£45,£208
6,Rome2rio,Berlin,London,0,1,0,0,618,£144,£523
7,Rome2rio,Berlin,London,0,1,0,0,696,£154,£435
8,Rome2rio,Berlin,London,1,0,0,0,1025,£27,£42
9,Rome2rio,Berlin,London,2,0,0,0,1345,£47,£97


liste_capitale = ["Paris",
                  "Berlin",
                  "Rome",
                  "Madrid",
                  "London",
                  "Dublin",
                  "Lisbon",
                  "Brussels",
                  "Luxembourg",
                  "Amsterdam",
                  "Bern",
                  "Copenhagen",
                  "Oslo",
                  "Stockholm",
                  "Helsinki",
                  "Tallinn",
                  "Riga",
                  "Vilnius",
                  "Warsaw",
                  "Prague",
                  "Vienna",
                  "Bratislava",
                  "Budapest",
                  "Ljubljana",
                  "Ankara"
                  "Bucharest",
                  "Belgrade",
                  "Sofia",
                  "Tijana",
                  "Skopje",
                  "Athens",
                  "Chisinau",
                  "Kiev",
                  "Minsk",
                  "Moscow",
                  "Tbilissi",
                  "Bakou",
                  "Verevan",
                  "Sarajevo",
                  "Reykjavik",
                  "Valletta",
                  "Zagreb",
                  "Nicosia",
                  "Andorra la Vella",
                  "San Marino",
                  "Vatican City"]

In [6]:
example_capitale = ["Paris",
                  "Berlin",
                  "Rome",
                  "Madrid",
                  "London",
                  "Dublin",
                  "Lisbon"]

In [7]:
# To run the function getdata to a list of capitale
tmp = [[x, y] for x in example_capitale for y in example_capitale if x != y]
appended_data = []
for i in range(len(tmp)):
    appended_data.append(getdata(tmp[i][0], tmp[i][1]))
appended_data = pd.concat(appended_data, ignore_index=True)

In [8]:
appended_data

,Data_Source,Departure_city,Arrival_city,Nb_bus_taken,Nb_train_taken,Nb_car_taken,Nb_plane_taken,Duration,Price_min,Price_max
0,Rome2rio,Paris,Berlin,0,0,0,1,285,47€,224€
1,Rome2rio,Paris,Berlin,0,0,0,1,281,49€,277€
2,Rome2rio,Paris,Berlin,0,0,0,1,293,57€,326€
3,Rome2rio,Paris,Berlin,0,0,0,1,438,74€,399€
4,Rome2rio,Paris,Berlin,0,1,0,1,339,117€,348€
5,Rome2rio,Paris,Berlin,0,1,0,0,509,187€,277€
6,Rome2rio,Paris,Berlin,0,1,0,0,743,90€,140€
7,Rome2rio,Paris,Berlin,1,0,0,0,855,45€,65€
8,Rome2rio,Paris,Berlin,1,0,0,0,989,37€,45€
9,Rome2rio,Paris,Berlin,0,0,1,0,576,120€,190€


In [9]:
# The others we want to have in the final dataframe
col_names = ['Date_Review', 'Review', 'Airline_Name', 'Airline_Type', 'Region_Operation', 'Aircraft_Type', 'Cabin_Class', 'Type_Of_Lounge',
             'Type_Of_Traveller', 'Date_Visit', 'Date_Flown', 'Airport', 'Route', 'Category', 'Category_Detail',
             'Cabin_Staff_Service', 'Lounge_Staff_Service', 'Bar_And_Beverages', 'Food_And_Beverages', 'Ground_Service', 'Catering', 'Cleanliness',
             'Lounge_Comfort', 'Aisle_Space', 'Wifi_And_Connectivity', 'Inflight_Entertainment', 'Viewing_Tv_Screen', 'Power_Supply',
             'Seat', 'Seat_type', 'Seat_Comfort', 'Seat_Legroom', 'Seat_Storage', 'Seat_Width', 'Seat_Recline', 'Washrooms',
             'Value_For_Money', 'Overall_Customer_Rating', 'Overall_Service_Rating', 'Overall_Airline_Rating',
             'Recommended', 'Title', 'Author', 'Description', 'Date_publication',
             'View_count', 'Likes', 'Dislikes', 'Nb_subscribers', 'Nb_comments', 'Nb_sharing', 'Hashtags', 'Awards', 'Registration', 'Language']
datafinal = pd.DataFrame(columns=col_names)

In [10]:
# Cocatenate to have all features
rom2rio_ = pd.concat([appended_data, datafinal], sort=False)

In [11]:
# Export the final result to json
rom2rio = rom2rio_.to_json()
with open('data_rom2rio.json', 'a', encoding='utf8') as outfile:
    json.dump(rom2rio, outfile, ensure_ascii=False, indent=4)